A simple movie recommendation systems using the [MovieLens dataset](https://www.kaggle.com/prajitdatta/movielens-100k-dataset/data). The original source of the data is [here](https://grouplens.org/datasets/movielens/).

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import DBSCAN, KMeans
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option("display.max_colwidth", 0)

## Read Data

In [3]:
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(
    os.path.join("data", "ml-100k", "u.data"),
    sep="\t",
    names=r_cols,
    encoding="latin-1",
)
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
user_key = "user_id"
item_key = "movie_id"

### 2.1 Terminology

**Constants**:

 - $N$: the number of users, indexed by $n$
 - $M$: the number of movies, indexed by $m$
 - $\mathcal{R}$: the set of indices $(n,m)$ where we have ratings in the utility matrix $Y$
    - Thus $|\mathcal{R}|$ is the total number of ratings
 
**The data**:

 - $Y$: the utility matrix containing ratings, contains a lot of missing entries
 - `train_mat` and `valid_mat`: Utility matrices for train and validation sets, respectively

In [5]:
N = ratings.shape[0]
M = len(np.unique(np.array(ratings[["movie_id"]])))

print(N)
print(M)

100000
1682


**Note**: The fraction of non-missing ratings in $Y$ is $\frac{|\mathcal{R}|}{NM}$, where $NM$ is the total number of entries $|Y|$ in the utility matrix.

<br><br>

### Splitting the data

Using a `random_state` so that results are consistent between users. 

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = ratings.copy()
X_train, X_valid = train_test_split(X, test_size=0.2, random_state=42)

### Utility matrix 

In [8]:
user_mapper = dict(zip(np.unique(ratings[user_key]), list(range(N))))
item_mapper = dict(zip(np.unique(ratings[item_key]), list(range(M))))
user_inverse_mapper = dict(zip(list(range(N)), np.unique(ratings[user_key])))
item_inverse_mapper = dict(zip(list(range(M)), np.unique(ratings[item_key])))

In [9]:
# As per lecture notes
def Construct_Y(data, N, M):
    Y = np.zeros((N,M))
    Y.fill(np.nan)
    for index, val in data.iterrows():
        n = user_mapper[val[user_key]]
        m = item_mapper[val[item_key]]
        Y[n,m] = val["rating"]
        
    return Y

train_mat = Construct_Y(X_train, N, M)
valid_mat = Construct_Y(X_valid, N, M)

In [10]:
train_mat.shape

(100000, 1682)

The train and validation matrices have the same size, but they are created from the train and validation sets. In other words, for the validation matrix, 20% of the rows (the ones coming from the validation sets) will potentially have non-zero entries for the features, but all the other rows will have missing values.

<br><br>

### Evaluation and baseline

The `error` function returns RMSE. The `evaluate` function prints the train and validation RMSEs. 

In [11]:
def error(Y1, Y2):
    """
    Returns the root mean squared error (RMSE).
    """
    return np.sqrt(np.nanmean((Y1 - Y2) ** 2))


def evaluate(pred_Y, train_mat, valid_mat, model_name="Global average"):
    print("%s train RMSE: %0.2f" % (model_name, error(pred_Y, train_mat)))
    print("%s valid RMSE: %0.2f" % (model_name, error(pred_Y, valid_mat)))

**Note:** When we evaluate recommender systems, we are calculating train error and validation error (in this case, using the matric RMSE). First, we take then train_mat and apply our model to it. Then to calculate train error, we compare the model's prediction to the train_mat portion of the ratings. For validation error, we compare our model's predicted rating to the valid_mat portion of the ratings. 

<br>

In [12]:
class Global_Baseline:
    
    def __init__(self):
        self.N = None
        self.M = None
        self.means = None
        
    def fit(self, X):
        N, M = X.shape
        pred = np.zeros((N,M))
        for j in range(M):
            column_mean = np.nanmean(X[:,j])
            pred[:,j] = [column_mean for inp in range(N)]
        self.means = pred

In [13]:
gb = Global_Baseline()
gb.fit(train_mat)

/var/folders/qp/hky_8qn94bb4xlp8wx5fclrm0000gn/T/ipykernel_57426/3881244217.py:12: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(X[:,j])


In [14]:
pred_Y = gb.means
evaluate(pred_Y, train_mat, valid_mat)

Global average train RMSE: 1.00
Global average valid RMSE: 1.02


In [15]:
pd.DataFrame(pred_Y).head(5)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,3.86533,3.203883,3.101449,3.549133,3.246377,3.647059,3.811912,3.936047,3.90795,3.891892,...,3.0,NaN,NaN,2.0,NaN,1.0,3.0,2.0,3.0,NaN
1,3.86533,3.203883,3.101449,3.549133,3.246377,3.647059,3.811912,3.936047,3.90795,3.891892,...,3.0,NaN,NaN,2.0,NaN,1.0,3.0,2.0,3.0,NaN
2,3.86533,3.203883,3.101449,3.549133,3.246377,3.647059,3.811912,3.936047,3.90795,3.891892,...,3.0,NaN,NaN,2.0,NaN,1.0,3.0,2.0,3.0,NaN
3,3.86533,3.203883,3.101449,3.549133,3.246377,3.647059,3.811912,3.936047,3.90795,3.891892,...,3.0,NaN,NaN,2.0,NaN,1.0,3.0,2.0,3.0,NaN
4,3.86533,3.203883,3.101449,3.549133,3.246377,3.647059,3.811912,3.936047,3.90795,3.891892,...,3.0,NaN,NaN,2.0,NaN,1.0,3.0,2.0,3.0,NaN


<br><br>

### Collaborative Filtering

Using the [`surprise`](https://surprise.readthedocs.io/en/stable/) package which has an implementation of the SVD algorithm for collaborative filtering. You can install it as follows:

```
>> conda install -c conda-forge scikit-surprise
or 
>> pip install scikit-surprise
```

In [16]:
import surprise
from surprise import SVD, Dataset, Reader, accuracy

In [17]:
ratings.head(2)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [18]:
reader = Reader()
data = Dataset.load_from_df(ratings.drop("timestamp", axis=1), reader)

train, valid = surprise.model_selection.train_test_split(data, test_size = 0.2)

k = 10
algo = SVD(n_factors = k)
algo.fit(train)
svd_pred = algo.test(valid)
accuracy.rmse(svd_pred, verbose=True)

RMSE: 0.9392


0.9392083167432567

**Remark:** The RMSE has improved compared to the global baseline, which was 1.02 for validation.